## TOC
1. LB1: Import libraries
2. 

In [1]:
# LB1
import datetime

import numpy as np
import pandas_datareader.data as web
import yfinance as yfin

yfin.pdr_override()

In [2]:
# IP1: Pulling data from Yahoo

start = datetime.date.today() - datetime.timedelta(days=5*365)
end = datetime.date.today()
# start = datetime.date(2016, 11, 16)
# end = datetime.date(2021, 11, 19)
df = web.DataReader(["AMZN", "F", "BTC-USD"], start, end)["Adj Close"]
df["AMZN"] = 20 * df["AMZN"]  # code adapted for recent AMZN stock split

[*********************100%***********************]  3 of 3 completed


In [3]:
# IP2: Look at data head

df.head(10)

,AMZN,BTC-USD,F
Date,,,
2018-06-24,NaN,6173.229980,NaN
2018-06-25,1663.150024,6249.180176,9.196166
2018-06-26,1691.089935,6093.669922,9.212160
2018-06-27,1660.509949,6157.129883,9.132193
2018-06-28,1701.450043,5903.439941,9.020238
2018-06-29,1699.799957,6218.299805,8.852307
2018-06-30,NaN,6404.000000,NaN
2018-07-01,NaN,6385.819824,NaN
2018-07-02,1713.780060,6614.180176,8.876301


In [4]:
# IP3: Look at data overall

df.describe()

,AMZN,BTC-USD,F
count,1257.000000,1825.000000,1257.000000
mean,2452.185185,21857.545490,10.072764
std,663.532441,16661.299483,3.467641
min,1343.959961,3236.761719,3.589956
25%,1847.839966,8181.390137,7.569422
50%,2271.000061,16847.755859,8.857571
75%,3162.009888,33746.003906,12.329062
max,3731.409912,67566.828125,22.664917


In [6]:
# CR1: Calculating Simple Return Buy and Hold
# Using initial money to buy at past timestamp. Quantity = Initial Money / Price
# Multiply Quantity with current price -> Simple Return
np.round(df.tail(3), 3)

,AMZN,BTC-USD,F
Date,,,
2023-06-20,2515.6,28327.488,14.22
2023-06-21,2496.6,30027.297,14.02
2023-06-22,2603.0,29912.281,14.19


In [7]:
# CR1: Calculating Simple Return Buy and Hold
print("AMZN = ", 1.282 * 3696.06)
print("F = ", 114.863 * 17.82)
print("Bitcoin = ", 1.353 * 56942.14)

AMZN =  4738.34892
F =  2046.85866
Bitcoin =  77042.71542


In [8]:
# CR2: Pulling data from Yahoo, then join with current dataframe
df = df.join(web.DataReader(["BLV"], start, end)["Adj Close"])
df.rename(columns={"Adj Close": "BLV"}, inplace=True)

[*********************100%***********************]  1 of 1 completed


In [9]:
# CR3: Drop null value, remove unsused columns, and calculating log return
df = df.dropna()
df["Amazon"] = np.log(df.AMZN) - np.log(df.AMZN.shift(1))
df["Ford"] = np.log(df.F) - np.log(df.F.shift(1))
df["Bitcoin"] = np.log(df["BTC-USD"]) - np.log(df["BTC-USD"].shift(1))
df = df.iloc[1:, 3:]

In [10]:
df.describe()

,BLV,Amazon,Ford,Bitcoin
count,1256.000000,1256.000000,1256.000000,1256.000000
mean,85.210584,0.000357,0.000345,0.001247
std,10.592206,0.022970,0.026954,0.044328
min,65.101273,-0.151398,-0.131524,-0.464730
25%,74.721025,-0.011141,-0.013636,-0.017065
50%,86.600281,0.001075,0.000000,0.000864
75%,95.853260,0.012016,0.013974,0.021047
max,103.712120,0.126949,0.210596,0.203046


In [12]:
# CR4: Converting Daily Return to Annual Return (percent)
# In this case, use the mean of return in 5 years, then convert into annual return
((df[["Bitcoin", "Ford"]].mean() + 1).pow(365) - 1) * 100

Bitcoin    57.578181
Ford       13.431442
dtype: float64